In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_WARNING = 0.9
THRESH_SHOCK_INDEX_CRITICAL = 1.0
THRESH_MAP_LOW = 65  # Mean Arterial Pressure threshold for shock
THRESH_LACTATE_HIGH = 2.0  # mmol/L for tissue hypoperfusion
THRESH_LACTATE_CRITICAL = 4.0
THRESH_UO_LOW = 0.5  # mL/kg/hr (oliguria)

# Trend Analysis
TREND_WINDOW = 6  # Number of readings for short-term trend analysis

THRESH_RR_ACS = 22
THRESH_HR_ACS = 100
BP_DROP_PCT = 0.20      # drop >= 20% considered significant
BP_REBOUND_PCT = 0.10   # rebound >= 10% of the nadir considered rebound
PERSIST_CONSECUTIVE = 3 # number of consecutive readings to consider persistent
ACS_WINDOW_MINUTES = 120
# AGE-SPECIFIC VITAL SIGN THRESHOLDS (Low, Normal, High)

AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30, 'rr_normal': 40, 'rr_high': 60,
        'hr_low': 100, 'hr_normal': 140, 'hr_high': 160,
        'sbp_low': 60, 'sbp_normal': 70, 'sbp_high': 90,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24, 'rr_normal': 30, 'rr_high': 40,
        'hr_low': 80, 'hr_normal': 120, 'hr_high': 140,
        'sbp_low': 70, 'sbp_normal': 85, 'sbp_high': 100,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'child': {
        'rr_low': 16, 'rr_normal': 20, 'rr_high': 30,
        'hr_low': 70, 'hr_normal': 90, 'hr_high': 110,
        'sbp_low': 80, 'sbp_normal': 95, 'sbp_high': 110,
        'temp_low': 36.0, 'temp_normal': 37.0, 'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 75, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 105, 'sbp_high': 120,
        'temp_low': 35.8, 'temp_normal': 36.8, 'temp_high': 37.8
    },
    'adult': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 80, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 115, 'sbp_high': 130,
        'temp_low': 35.5, 'temp_normal': 36.8, 'temp_high': 38.0
    },
    'geriatric': {  # Added for elderly patients who may have different baselines
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 24, # Often higher RR baseline
        'hr_low': 55, 'hr_normal': 70, 'hr_high': 90, # Often lower HR
        'sbp_low': 90, 'sbp_normal': 125, 'sbp_high': 140, # Often higher SBP
        'temp_low': 35.5, 'temp_normal': 36.5, 'temp_high': 37.5 # Often lower temp
    }
}


SCORE_THRESHOLDS = {
    "hypotensive_shock": {  
        "yellow": 3,   # Monitor/Review
        "orange": 6,   # Urgent
        "red": 9       # Critical
    },
}





In [3]:
def assign_age_category(df):
    """
    Assigns an age category based on the 'age' column in the DataFrame.
    Now includes more granular pediatric categories and a geriatric category.
    """
    df = df.copy()
    
    def _categorize(age):
        if age <= 0.083: return 'neonate'     # < 1 month
        elif age <= 1:   return 'infant'      # 1 month - 1 year
        elif age < 5:    return 'child'       # 1 - 5 years
        elif age < 13:   return 'adolescent'  # 5 - 12 years
        elif age < 65:   return 'adult'       # 13 - 64 years
        else:            return 'geriatric'   # 65+ years
    
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_categorize)
    else:
        # Default to adult if age is not provided
        df['age_category'] = 'adult'
    
    return df

In [4]:
def apply_vital_range_flags(df):
    """
    Applies age-specific thresholds to flag abnormal vital signs.
    Now includes flags for all parameters needed across pipelines.
    Automatically normalizes column names so pipeline always works.
    """
    df = df.copy()
    
    # ✅ Normalize column names
    df = df.rename(columns={
        'SpO2': 'spo2',
        'SBP': 'sbp',
        'Temp': 'temperature',
        'HR': 'heart_rate',
        'RR': 'resp_rate'
    })

    df = assign_age_category(df)  # Inject age group

    # SpO₂ flags (Absolute threshold)
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW
    df['flag_spo2_critical'] = df['spo2'] < THRESH_SPO2_CRITICAL

    # Temperature flags
    df['flag_temp_high'] = df.apply(
        lambda row: row['temperature'] >= AGE_THRESHOLDS[row['age_category']]['temp_high'], axis=1
    )
    df['flag_temp_low'] = df.apply(
        lambda row: row['temperature'] < AGE_THRESHOLDS[row['age_category']]['temp_low'], axis=1
    )

    # Respiratory Rate flags
    df['flag_rr_low'] = df.apply(
        lambda row: row['resp_rate'] < AGE_THRESHOLDS[row['age_category']]['rr_low'], axis=1
    )
    df['flag_rr_high'] = df.apply(
        lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1
    )

    # Heart Rate flags
    df['flag_hr_low'] = df.apply(
        lambda row: row['heart_rate'] < AGE_THRESHOLDS[row['age_category']]['hr_low'], axis=1
    )
    df['flag_hr_high'] = df.apply(
        lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1
    )

    # Calculate Shock Index (handle division by zero)
    df['shock_index'] = df['heart_rate'] / np.clip(df['sbp'], a_min=1, a_max=None)    

    # Flag based on Shock Index
    df['flag_si_warning'] = df['shock_index'] >= THRESH_SHOCK_INDEX_WARNING
    df['flag_si_critical'] = df['shock_index'] >= THRESH_SHOCK_INDEX_CRITICAL

    # Blood Pressure flags
    df['flag_sbp_low'] = df.apply(
        lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1
    )
    df['flag_sbp_high'] = df.apply(
        lambda row: row['sbp'] >= AGE_THRESHOLDS[row['age_category']]['sbp_high'], axis=1
    )
    df['flag_dbp_low'] = df.apply(
        lambda row: row['dbp'] < (AGE_THRESHOLDS[row['age_category']]['sbp_low'] * 0.6), axis=1
    ) # Estimate DBP low
    df['flag_dbp_high'] = df.apply(
        lambda row: row['dbp'] >= (AGE_THRESHOLDS[row['age_category']]['sbp_high'] * 0.6), axis=1
    ) # Estimate DBP high

    return df


In [5]:
def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_WARNING:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends

In [6]:
def _bp_drop_then_rebound(sbp_series, times_seconds=None, drop_pct=BP_DROP_PCT, rebound_pct=BP_REBOUND_PCT):
    """
    Detect 'drop then rise' BP instability pattern in a 1-D sbp_series (np.array or pd.Series).
    Returns (flag_bool, details_dict)
    details contains: {'max_before_nadir':..., 'nadir':..., 'last_value':..., 'drop_frac':..., 'rebound_frac':...}
    """
    vals = np.array(sbp_series, dtype=float)
    details = {'max_before_nadir': None, 'nadir': None, 'last_value': None, 'drop_frac': 0.0, 'rebound_frac': 0.0}
    if len(vals) < 3 or np.all(np.isnan(vals)):
        return False, details

    # remove NaNs for pattern analysis but keep indices mapping
    valid_idx = ~np.isnan(vals)
    if valid_idx.sum() < 3:
        return False, details
    v = vals[valid_idx]

    # locate global nadir (first minimal occurrence)
    nadir_idx = np.argmin(v)
    nadir = v[nadir_idx]

    # max before nadir (use values prior to nadir if available, else overall max)
    if nadir_idx > 0:
        max_before = np.max(v[:nadir_idx])
    else:
        max_before = np.max(v)

    last_val = v[-1]

    drop_frac = (max_before - nadir) / max_before if max_before > 0 else 0.0
    rebound_frac = (last_val - nadir) / max_before if max_before > 0 else 0.0

    details.update({'max_before_nadir': float(max_before),
                    'nadir': float(nadir),
                    'last_value': float(last_val),
                    'drop_frac': float(round(drop_frac, 3)),
                    'rebound_frac': float(round(rebound_frac, 3))})

    flag = (drop_frac >= drop_pct) and (rebound_frac >= rebound_pct)
    return bool(flag), details


In [7]:
def _consecutive_count(series_bool):
    """Return max consecutive True count in boolean series (list/np.array/pd.Series)."""
    arr = np.array(series_bool, dtype=bool)
    if arr.size == 0:
        return 0
    maxc = 0
    cur = 0
    for v in arr:
        if v:
            cur += 1
            if cur > maxc:
                maxc = cur
        else:
            cur = 0
    return int(maxc)


In [8]:
def detect_acs_concern(df_patient,
                       window_minutes=ACS_WINDOW_MINUTES,
                       persist_required=PERSIST_CONSECUTIVE,
                       rr_thresh=THRESH_RR_ACS,
                       hr_thresh=THRESH_HR_ACS,
                       bp_drop_pct=BP_DROP_PCT,
                       bp_rebound_pct=BP_REBOUND_PCT):
    """
    Detect Acute Coronary Syndrome concern from vitals.
    df_patient: DataFrame for single patient. Must contain at least:
        ['timestamp' (datetime), 'resp_rate', 'heart_rate', 'sbp'].
      Optional: 'spo2', 'chest_pain_flag', 'age' or 'age_category' (used by your other functions)
    Returns a dict:
      {
        'action': 'red'/'orange'/'yellow'/'monitor',
        'reasons': [...],
        'flags': {...},
        'trend_summary': {...},   # from compute_recent_trends_delta if available
        'bp_instability_details': {...},
        'recommendation': '...'
      }
    """
    # Defensive copy & ordering
    df = df_patient.copy().sort_values('timestamp').reset_index(drop=True)
    if df.empty:
        return {'action': 'monitor', 'reasons': ['No data'], 'flags': {}, 'recommendation': 'No action'}

    now = df['timestamp'].max()
    window_start = now - pd.Timedelta(minutes=window_minutes)
    win = df[df['timestamp'] >= window_start].copy()
    if win.empty:
        win = df.tail( min(len(df), 10) ).copy()  # fallback to last few if window empty

    latest = win.iloc[-1]
    hr = latest.get('heart_rate', np.nan)
    rr = latest.get('resp_rate', np.nan)
    sbp = latest.get('sbp', np.nan)
    spo2 = latest.get('spo2', np.nan)
    chest = bool(latest.get('chest_pain_flag', False)) if 'chest_pain_flag' in win.columns else False

    # Basic flags
    flag_rr_high = (not np.isnan(rr)) and (rr >= rr_thresh)
    flag_hr_high = (not np.isnan(hr)) and (hr >= hr_thresh)
    flag_concerning_combo = flag_rr_high and flag_hr_high

    # shock index (protect against zero sbp)
    si = hr / np.clip(sbp, a_min=1, a_max=None) if (not np.isnan(hr) and not np.isnan(sbp)) else np.nan
    flag_si_warn = (not np.isnan(si)) and (si >= THRESH_SHOCK_INDEX_WARNING)
    flag_si_critical = (not np.isnan(si)) and (si >= THRESH_SHOCK_INDEX_CRITICAL)
    flag_hypotension = (not np.isnan(sbp)) and (sbp < AGE_THRESHOLDS.get(latest.get('age_category','adult'), AGE_THRESHOLDS['adult'])['sbp_low'])

    # Persistence: how many consecutive recent readings meet RR>= & HR>= simultaneously?
    combo_series = (win.get('resp_rate', np.nan) >= rr_thresh) & (win.get('heart_rate', np.nan) >= hr_thresh)
    max_consec = _consecutive_count(combo_series.fillna(False).values)

    # BP instability: drop then rebound detection on SBP values in window
    bp_instability_flag, bp_details = _bp_drop_then_rebound(win.get('sbp', pd.Series(dtype=float)).values,
                                                            drop_pct=bp_drop_pct, rebound_pct=bp_rebound_pct)

    # Call existing trend function if available (non-fatal import)
    trend_summary = {}
    try:
        trend_summary = compute_recent_trends_delta(win)  # uses your existing function
    except Exception:
        trend_summary = {}

    # Decision logic (tiered)
    action = 'monitor'
    reasons = []

    # Red criteria (immediate)
    if flag_si_critical:
        action = 'red'
        reasons.append('Critical shock index (>= {})'.format(THRESH_SHOCK_INDEX_CRITICAL))
    if flag_hypotension and chest:
        action = 'red'
        reasons.append('Hypotension with chest pain')
    if bp_instability_flag and chest:
        action = 'red'
        reasons.append('BP drop-then-rebound pattern + chest pain (unstable BP)')

    # Orange criteria (high suspicion -> urgent ECG + troponin)
    if action != 'red':
        if flag_concerning_combo and chest:
            action = 'orange'
            reasons.append('RR >= {} & HR >= {} with chest pain'.format(rr_thresh, hr_thresh))
        elif flag_concerning_combo and flag_si_warn:
            action = 'orange'
            reasons.append('RR>= {} & HR>= {} with elevated shock index'.format(rr_thresh, hr_thresh))
        elif bp_instability_flag and max_consec >= 1:  # unstable BP even without chest pain
            action = 'orange'
            reasons.append('BP instability pattern detected')

    # Yellow criteria (observe + expedited ECG if persists)
    if action == 'monitor':
        if flag_concerning_combo and max_consec >= persist_required:
            action = 'yellow'
            reasons.append('Persistent RR>= {} & HR>= {} for {} consecutive readings'.format(rr_thresh, hr_thresh, persist_required))
        elif flag_concerning_combo:
            action = 'yellow'
            reasons.append('Transient RR>= {} & HR>= {} (monitor/obtain ECG if persists)'.format(rr_thresh, hr_thresh))
        elif flag_hr_high or flag_rr_high:
            action = 'yellow'
            reasons.append('Either HR or RR elevated (monitor and repeat vitals)')

    # Safety override: extreme hypoxia or syncope columns if present -> escalate to red
    if ('spo2' in win.columns and not np.isnan(spo2) and spo2 <= 88) or ('syncope_flag' in win.columns and win['syncope_flag'].iloc[-1]):
        action = 'red'
        reasons.append('Severe hypoxia or recent syncope - escalate')

    # Compose flags dict
    flags = {
        'rr_high': bool(flag_rr_high),
        'hr_high': bool(flag_hr_high),
        'concerning_combo': bool(flag_concerning_combo),
        'max_consecutive_combo': int(max_consec),
        'bp_instability': bool(bp_instability_flag),
        'shock_index': float(round(si, 3)) if not np.isnan(si) else None,
        'si_warning': bool(flag_si_warn),
        'si_critical': bool(flag_si_critical),
        'hypotension': bool(flag_hypotension),
        'chest_pain_flag': bool(chest)
    }

    rec = "If action is red/orange: obtain 12-lead ECG immediately and order high-sensitivity troponin (serial 0 and 1-3h per local protocol). For yellow: repeat vitals and get ECG if persists."

    return {
        'action': action,
        'reasons': reasons,
        'flags': flags,
        'trend_summary': trend_summary,
        'bp_instability_details': bp_details,
        'recommendation': rec
    }

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Generate synthetic test data
np.random.seed(0)

timestamps = [datetime(2025, 9, 17, 8, 0) + timedelta(minutes=30*i) for i in range(12)]

data = {
    "timestamp": timestamps,
    "SpO2": [97, 96, 95, 93, 90, 87, 85, 84, 83, 82, 92, 95],  # drop to simulate desaturation
    "heart_rate":   [78, 80, 82, 85, 90, 105, 110, 120, 125, 130, 95, 85],  # tachycardia episode
    "resp_rate":   [18, 19, 20, 22, 24, 28, 30, 32, 34, 35, 22, 18],  # increasing RR
    "SBP":  [120, 118, 116, 110, 105, 98, 92, 88, 85, 82, 100, 115],  # dropping SBP
    "Temp": [36.7, 36.6, 36.8, 37.0, 37.2, 37.5, 37.6, 37.7, 37.6, 37.4, 36.9, 36.8]  # mild fever pattern
}



df_test = pd.DataFrame(data)
print(df_test)
result=detect_acs_concern(df_test)
print(result)

             timestamp  SpO2  heart_rate  resp_rate  SBP  Temp
0  2025-09-17 08:00:00    97          78         18  120  36.7
1  2025-09-17 08:30:00    96          80         19  118  36.6
2  2025-09-17 09:00:00    95          82         20  116  36.8
3  2025-09-17 09:30:00    93          85         22  110  37.0
4  2025-09-17 10:00:00    90          90         24  105  37.2
5  2025-09-17 10:30:00    87         105         28   98  37.5
6  2025-09-17 11:00:00    85         110         30   92  37.6
7  2025-09-17 11:30:00    84         120         32   88  37.7
8  2025-09-17 12:00:00    83         125         34   85  37.6
9  2025-09-17 12:30:00    82         130         35   82  37.4
10 2025-09-17 13:00:00    92          95         22  100  36.9
11 2025-09-17 13:30:00    95          85         18  115  36.8
{'action': 'monitor', 'reasons': [], 'flags': {'rr_high': False, 'hr_high': False, 'concerning_combo': False, 'max_consecutive_combo': 3, 'bp_instability': False, 'shock_index': Non

In [10]:
np.random.seed(42)

def generate_synthetic_vitals(n=20, scenario="acs"):
    timestamps = [datetime.now() - timedelta(minutes=5*i) for i in range(n)][::-1]
    data = {
        "timestamp": timestamps,
        "age": [55]*n,  # Adult
    }

    if scenario == "normal":
        data.update({
            "rr": np.random.randint(14, 18, n),
            "hr": np.random.randint(70, 90, n),
            "sbp": np.random.randint(110, 130, n),
            "spo2": np.random.randint(95, 99, n),
            "temperature": np.random.uniform(36.5, 37.2, n),
            "chest_pain_flag": [False]*n
        })
    elif scenario == "acs":
        # Simulate tachypnea + tachycardia + unstable BP + chest pain
        rr = np.random.randint(22, 28, n)
        hr = np.random.randint(100, 120, n)
        sbp = np.linspace(130, 90, n//2).tolist() + np.linspace(90, 140, n//2).tolist()
        data.update({
            "rr": rr,
            "hr": hr,
            "sbp": sbp,
            "spo2": np.random.randint(90, 95, n),
            "temperature": np.random.uniform(36.8, 37.5, n),
            "chest_pain_flag": [False]*(n-3) + [True]*3
        })
    elif scenario == "shock_index":
        # High HR, low SBP scenario
        rr = np.random.randint(20, 26, n)
        hr = np.random.randint(120, 140, n)
        sbp = np.random.randint(70, 90, n)
        data.update({
            "rr": rr,
            "hr": hr,
            "sbp": sbp,
            "spo2": np.random.randint(85, 90, n),
            "temperature": np.random.uniform(37.0, 38.0, n),
            "chest_pain_flag": [True]*n
        })
    else:
        raise ValueError("Unknown scenario")

    return pd.DataFrame(data)

# Create datasets
df_normal = generate_synthetic_vitals(20, "normal")
df_acs = generate_synthetic_vitals(20, "acs")
df_shock = generate_synthetic_vitals(20, "shock_index")

import pandas as pd
dfs_preview = {
    "normal_case": df_normal.head(),
    "acs_case": df_acs.head(),
    "shock_case": df_shock.head(),
}
dfs_preview

{'normal_case':                    timestamp  age  rr  hr  sbp  spo2  temperature  \
 0 2025-11-08 14:49:13.959749   55  16  71  116    96    36.561945   
 1 2025-11-08 14:54:13.959748   55  17  81  127    96    36.637188   
 2 2025-11-08 14:59:13.959747   55  14  75  113    96    36.531659   
 3 2025-11-08 15:04:13.959745   55  16  71  123    96    36.727731   
 4 2025-11-08 15:09:13.959744   55  16  70  127    96    36.772074   
 
    chest_pain_flag  
 0            False  
 1            False  
 2            False  
 3            False  
 4            False  ,
 'acs_case':                    timestamp  age  rr   hr         sbp  spo2  temperature  \
 0 2025-11-08 14:49:13.961227   55  22  104  130.000000    94    36.885462   
 1 2025-11-08 14:54:13.961226   55  24  114  125.555556    92    37.049408   
 2 2025-11-08 14:59:13.961225   55  26  113  121.111111    94    37.434780   
 3 2025-11-08 15:04:13.961224   55  23  102  116.666667    93    36.990493   
 4 2025-11-08 15:09:13.96122